In [82]:
import pandas as pd
from datetime import datetime
import clientprocessing as cp

In [83]:
clients={}
client_ids, df=cp.get_clients(give_df=True)

for client_id in client_ids:
    client_id=str(client_id)
    clients.update(dict({client_id:{}}))

print(clients)

{'1352108802': {}, '89281547': {}, '339825126': {}, '742770953': {}, '377235927': {}, '182523248': {}, '730912930': {}, '1890430896': {}, '146585525': {}, '1434827787': {}, '1588923963': {}, '1914596938': {}, '1991713172': {}, '930615366': {}, '962653': {}, '457842566': {}, '722889138': {}, '3045573': {}, '160484': {}, '141188634': {}, '1747449748': {}, '280972987': {}, '1343786340': {}, '1842186254': {}, '20028': {}, '1371350591': {}, '1094173963': {}, '2038648932': {}, '247049371': {}, '1222305620': {}, '44548903': {}, '1774320619': {}, '60253197': {}, '1039272198': {}, '232323': {}, '1999564604': {}, '1700828332': {}, '2024025171': {}, '1683841519': {}, '1444792769': {}, '883524406': {}, '360285558': {}, '470065716': {}, '1887255116': {}, '674982001': {}, '1348048562': {}, '1662358249': {}, '447268576': {}, '1167515548': {}, '204907870': {}, '601976963': {}, '957010757': {}, '2035193665': {}, '1288402200': {}, '32535': {}, '848061486': {}, '203634036': {}, '1816143665': {}, '5465034

In [106]:
for client_id in client_ids:
    client_specific_df = df[df["client_id"] == client_id][[
        "period_end", "source", "source_type", "attribution", "customers"]]

    periods = client_specific_df['period_end'].unique()
    
    # Update the dictionary for the client with periods
    period_dict = {}
    for period in periods:
        period_dict.update({period: {}})
        # Find the nr of attributed and unattributed customers
        period_df = client_specific_df[(
            client_specific_df['period_end'] == period)]
        attributed_customers = period_df[period_df['attribution']
                                         == 'ATTRIBUTED']['customers'].sum()
        unattributed_customers = period_df[period_df['attribution']
                                           != 'ATTRIBUTED']['customers'].sum()
        # Update the periods with the count of ATTRIBUTED and UNATTRIBUTED count
        # and the contribution rate for this period
        period_dict[period].update({"ATTRIBUTED": attributed_customers})
        period_dict[period].update({"UNATTRIBUTED": unattributed_customers})
        period_dict[period].update(
            {"CR": (attributed_customers/(attributed_customers+unattributed_customers)*100)})

        # Find and all the records to the period
        # period_dict[period].update({"records": {}})
        # record_dict={'source':None,'source_type':None,'attribution':None,'customers':None}
        # record_sources=period_df['source'].unique()
        records={}
        sources=period_df['source'].unique()
        for source in period_df['source'].unique():
            source_df = period_df[period_df['source'] == source]
            print(source_df)
            break
            source_attributed = len(source_df[source_df['attribution'] == 'ATTRIBUTED'])
            source_unattributed = len(source_df[source_df['attribution'] != 'ATTRIBUTED'])
            # Include 'source' key in the records dictionary
            records[source] = {
                'source': source,  # Include source name
                'source_type': source_df.iloc[0]['source_type'],
                'ATTRIBUTED': source_attributed,
                'UNATTRIBUTED': source_unattributed,
                'customers': source_df['customers'].sum()
            }

        period_dict[period].update({"records":records})
        # Add everything to the final dictionary
        clients[str(client_id)].update(period_dict)
        


                  period_end source source_type attribution  customers
0  2023-07-09T00:00:00-07:00   2034        link  ATTRIBUTED          1
                   period_end source source_type attribution  customers
23  2023-02-26T00:00:00-08:00   5617        link  ATTRIBUTED          1
                   period_end source source_type attribution  customers
27  2023-08-13T00:00:00-07:00   5262        link  ATTRIBUTED          8
                   period_end source source_type attribution  customers
31  2023-06-25T00:00:00-07:00   0411        link  ATTRIBUTED          1
                   period_end source source_type attribution  customers
34  2023-08-27T00:00:00-07:00   2712        link  ATTRIBUTED          1
                   period_end source source_type attribution  customers
39  2023-06-11T00:00:00-07:00   3105        link  ATTRIBUTED          9
                   period_end source source_type attribution  customers
40  2023-01-08T00:00:00-08:00   5619        link  ATTRIBUTED      

In [104]:
print(clients["2074305340"]["2023-02-12T00:00:00-08:00"]["records"]["direct"])

{'source': 'direct', 'source_type': 'link', 'ATTRIBUTED': 0, 'UNATTRIBUTED': 1, 'customers': 38}
